# Boston Neighborhoods Analysis

## Gather Neighborhood Data, Clean Data, and Visualize Data

### Import Relevant Libraries

In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

### Get data from excel file and load into a df

In [2]:
wb = load_workbook(filename='Boston Neighborhood Coordinates.xlsx', read_only=True)
ws = wb.active
col_names = [ws["A1"].value, ws["B1"].value, ws["C1"].value, ws["D1"].value]
header = np.array(col_names)

df = pd.DataFrame(ws.values, columns=header)
wb.close()

### Clean Data

In [3]:
df = df.drop(['Zip Code'], axis=1)
df = df.drop([0])
df.set_index('Neighborhood', inplace=True)

In [4]:
df.head()

,Latitude,Longitude
Neighborhood,,
Chinatown/Leather District,42.3515,-71.0602
West End,42.3653,-71.0646
South End,42.3378,-71.0706
Roxbury,42.3218,-71.0884
Mission Hill,42.3319,-71.0988


## Create Map of Neighborhoods

In [5]:
import folium # map rendering library

In [6]:
# create map of Boston using latitude and longitude values
map_Boston = folium.Map(location=[42.3261, -71.07], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df.index):
    label = f'{neighborhood}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Boston)
    
    folium.Circle([lat, lng],
                    radius=1000
                   ).add_to(map_Boston)
    
map_Boston

## Get Venue Data, Clean Data, and Visualize Data

In [7]:
import requests
import json # library to handle JSON files

In [8]:
CLIENT_ID = 'GKVJ5KSFU1RD00P24CV5JT2XSARMWSONFYRMLT2EK2IJFEBW'
CLIENT_SECRET = 'SSJALJ4S5QXDNQBZMNUNFBE1XMVQZETMJWU1JDXE2VWK2PZL'
VERSION = '20180605' # Foursquare API version

### Function that extracts the category of the venue

In [9]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Function that gets venues within a given radius of the Neighborhood

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        LIMIT = 1000
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        r = requests.get(url)
        print(r.status_code)
        results = r.json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
Boston_venues = getNearbyVenues(names=df.index, latitudes=df['Latitude'], longitudes=df['Longitude'], radius=1000)

Chinatown/Leather District
200
West End
200
South End
200
Roxbury
200
Mission Hill
200
Mattapan
200
South Boston
200
East Boston
200
Charlestown
200
Jamaica Plain
200
Roslindale
200
West Roxbury
200
Allston
200
Brighton
200
Hyde Park
200
North Cambridge 
200
Brookline Village
200
Chestnut Hill
200
Downtown
200
North End
200
Beacon Hill
200
Back Bay
200
Dorchester
200
Harvard Square 
200
East Cambridge
200
Fenway/Kenmore
200
Porter Square
200
West Cambridge
200
Inman Square
200
Central Square
200
Davis Square
200
Winter Hill
200
Assembly Square
200
Seaport
200


### Boston's Neighborhoods with Venues

In [12]:
print(Boston_venues.shape)
    
Boston_venues.nunique()

(2551, 7)


Neighborhood                34
Neighborhood Latitude       34
Neighborhood Longitude      31
Venue                     1892
Venue Latitude            2147
Venue Longitude           2148
Venue Category             292
dtype: int64

In [13]:
Boston_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chinatown/Leather District,42.3515,-71.0602,Gracenote Coffee,42.351657,-71.058204,Coffee Shop
1,Chinatown/Leather District,42.3515,-71.0602,Shabu Zen,42.350922,-71.060942,Asian Restaurant
2,Chinatown/Leather District,42.3515,-71.0602,Hostelling International Boston,42.351084,-71.063398,Hostel
3,Chinatown/Leather District,42.3515,-71.0602,Barry's Bootcamp,42.354010,-71.059776,Gym / Fitness Center
4,Chinatown/Leather District,42.3515,-71.0602,Tatte Bakery & Cafe,42.353111,-71.057117,Bakery


### Create Excel File with total venue counts per category

In [14]:
Venue_cat_count = Boston_venues.groupby('Venue Category').count()
Venue_cat_count.rename(columns={'Venue': 'Count'}, inplace=True)
Venue_cat_count = pd.DataFrame(Venue_cat_count['Count'], Venue_cat_count.index)
Venue_cat_count.to_excel('Venue_Category_Counts.xlsx')
Venue_cat_count.sort_values('Count', ascending=False)

,Count
Venue Category,
Pizza Place,96
Park,88
Coffee Shop,86
Italian Restaurant,80
Bakery,68
...,...
Baseball Stadium,1
Skate Park,1
Shopping Plaza,1


### Create Excel File with total venue counts per neighborhood

In [15]:
Venue_per_hood = Boston_venues.groupby('Neighborhood').count()
Venue_per_hood.rename(columns={'Venue': 'Count'}, inplace=True)
Venue_per_hood = pd.DataFrame(Venue_per_hood['Count'], Venue_per_hood.index)
Venue_per_hood.to_excel('Venue_Per_Hood.xlsx')
Venue_per_hood.sort_values('Count', ascending=False)

,Count
Neighborhood,
Allston,100
North End,100
Assembly Square,100
Harvard Square,100
Fenway/Kenmore,100
East Cambridge,100
Downtown,100
Porter Square,100
Davis Square,100


### Create dummy variables to proportion venue categories for each neighborhood

In [16]:
# one hot encoding
Boston_onehot = pd.get_dummies(Boston_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Boston_onehot['Neighborhood'] = Boston_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Boston_onehot.columns[-1]] + list(Boston_onehot.columns[:-1])
Boston_onehot = Boston_onehot[fixed_columns]

In [17]:
Boston_grouped = Boston_onehot.groupby('Neighborhood').mean().reset_index()
Boston_grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Allston,0.010000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,...,0.010000,0.000000,0.000000,0.020000,0.00,0.000000,0.00,0.000000,0.000000,0.00
1,Assembly Square,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.000000,0.000000,0.020000,0.000000,0.01,0.000000,0.00,0.000000,0.000000,0.00
2,Back Bay,0.010000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.01,0.030000,0.000000,0.01
3,Beacon Hill,0.010000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00
4,Brighton,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.012987,...,0.012987,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.012987,0.012987,0.00
5,Brookline Village,0.000000,0.019608,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.019608,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00
6,Central Square,0.020000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,...,0.040000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00
7,Charlestown,0.027027,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00
8,Chestnut Hill,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.034483,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00
9,Chinatown/Leather District,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,...,0.010000,0.000000,0.000000,0.010000,0.00,0.000000,0.01,0.010000,0.000000,0.00


### Get top 20 most occuring venues per neighborhood

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Boston_grouped['Neighborhood']

for ind in np.arange(Boston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Boston_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Allston,Korean Restaurant,Thai Restaurant,Chinese Restaurant,Mexican Restaurant,Pizza Place,Sushi Restaurant,Coffee Shop,Fried Chicken Joint,Falafel Restaurant,...,Bar,Taco Place,Rock Club,Japanese Restaurant,Café,Gastropub,Seafood Restaurant,Ice Cream Shop,Gift Shop,Vietnamese Restaurant
1,Assembly Square,Clothing Store,Donut Shop,Mexican Restaurant,Coffee Shop,American Restaurant,Restaurant,Sporting Goods Shop,Video Store,Ice Cream Shop,...,Park,Bar,Steakhouse,Brazilian Restaurant,Gym,Café,Sandwich Place,Seafood Restaurant,Italian Restaurant,Furniture / Home Store
2,Back Bay,Clothing Store,Seafood Restaurant,Spa,Park,Italian Restaurant,Coffee Shop,Gym,Bakery,Wine Shop,...,Mediterranean Restaurant,Shopping Mall,Plaza,Salad Place,Steakhouse,Ice Cream Shop,Hotel,Sushi Restaurant,Gym / Fitness Center,Jewelry Store
3,Beacon Hill,Coffee Shop,Italian Restaurant,Hotel,Historic Site,French Restaurant,Pizza Place,Seafood Restaurant,Park,Steakhouse,...,Boat or Ferry,Theater,Hotel Bar,Scenic Lookout,Bakery,Bar,Gym / Fitness Center,Market,Restaurant,Gourmet Shop
4,Brighton,Pizza Place,Chinese Restaurant,Bakery,Grocery Store,Sushi Restaurant,Pub,Convenience Store,Café,Donut Shop,...,Bank,Thai Restaurant,Pet Store,Deli / Bodega,Sandwich Place,Pharmacy,Greek Restaurant,Bar,Nail Salon,Dry Cleaner
5,Brookline Village,Park,Coffee Shop,Mexican Restaurant,Hotel,Pizza Place,Gastropub,Metro Station,Donut Shop,Liquor Store,...,Bar,Laundry Service,Sushi Restaurant,French Restaurant,Dive Bar,Salon / Barbershop,Sporting Goods Shop,Tapas Restaurant,Pet Service,Music Venue
6,Central Square,Coffee Shop,Vegetarian / Vegan Restaurant,New American Restaurant,Sandwich Place,Bakery,Italian Restaurant,Bar,Nightclub,Park,...,Cocktail Bar,Tea Room,Pizza Place,Pub,Brewery,Bookstore,Sushi Restaurant,Yoga Studio,Gym / Fitness Center,Tibetan Restaurant
7,Charlestown,Pizza Place,Gastropub,Café,Grocery Store,History Museum,Coffee Shop,National Park,Donut Shop,Theater,...,Bar,Yoga Studio,Lounge,Seafood Restaurant,Metro Station,Monument / Landmark,Moroccan Restaurant,Park,Thai Restaurant,Plaza
8,Chestnut Hill,Baseball Field,American Restaurant,Furniture / Home Store,Tennis Court,Italian Restaurant,Mobile Phone Shop,Grocery Store,Ice Cream Shop,Bank,...,Salon / Barbershop,Museum,Boutique,Coffee Shop,French Restaurant,Steakhouse,Forest,Food Truck,Clothing Store,Chinese Restaurant
9,Chinatown/Leather District,Bakery,Coffee Shop,Sandwich Place,Chinese Restaurant,Asian Restaurant,Park,Theater,Performing Arts Venue,Mexican Restaurant,...,Café,Bubble Tea Shop,Salad Place,Spa,Tea Room,Hotel,Seafood Restaurant,Historic Site,French Restaurant,Opera House


## Create Relevant Functions

### df_style Function

In [20]:
def df_style(val):
    boolean = 0
    for item in match:
        if val == item:
            boolean = 1
        
    if boolean == 1:
        return 'font-weight: bold'
    else:
        return ''

### Main_Score Function

In [21]:
def main_score(df):

    score = []
    j=0
    for row in df.itertuples():
        i=0 
        for item in zip(row):
            if item[0] == match[0]:
                if i <= 2:
                    score.append(1)
                elif i <= 4:
                    score.append(.9)
                elif i <= 6:
                    score.append(.75)
                elif i <= 11:
                    score.append(.5)
                elif i <= 16:
                    score.append(.33)
                elif i <= 21:
                    score.append(.2)
                else:
                    score.append(0)
            i=i+1
        j=j+1
        if j > len(score):
            score.append(0)
            
    df['Main Score'] = score
    df.sort_values(by=['Main Score'], ascending=False, inplace=True)

### Match Function

In [22]:
def match_func(criterion):

    match = []
    for item in criterion:

        mask = neighborhoods_venues_sorted.applymap(lambda x: item in str(x))
        match_results = neighborhoods_venues_sorted[mask == True].dropna(how='all')

        for i in range(0,len(match_results.index)):
            results = []
            results = match_results.iloc[i,].unique()
            match.append(results[1])
            
    match = list(dict.fromkeys(match))
    return match

### Match_Score Function

In [23]:
def match_score(df):

    num_match = []
    score = []

    for row in df.itertuples():
        i=0
        for item in zip(row):
            for crit in match:
                if item[0] == crit:
                    i=i+1
        num_match.append(i)
        score.append(i/5)
    df['Number of Matches'] = num_match
    df['Match Score'] = score
    df.sort_values(by=['Number of Matches'], ascending=False, inplace=True)

### Total_Score Function

In [24]:
def total_score(df):
    tot_score = []
    rows = len(df.index)
    for i in range(0,rows):
        main_score = df.loc[df.index[i], 'Main Score']
        match_score = df.loc[df.index[i], 'Match Score']
        tot_score.append((main_score + match_score)/2)
    df['Total Score'] = tot_score
    df.sort_values(by=['Total Score'], ascending=False, inplace=True)

## Run Analysis of Neighborhoods based on Criterion

### Create df that includes Neighborhoods with PRIORITY criterion

In [43]:
criterion = ['Italian', 'Pizza', 'Ice Cream', 'Gym', 'Coffee'] #First term is priority venue
match = []

mask = neighborhoods_venues_sorted.applymap(lambda x: criterion[0] in str(x))
search_priority = neighborhoods_venues_sorted[mask == True].dropna(how='all')

for i in range(0,len(search_priority.index)):
    results = []
    results = search_priority.iloc[i,].unique()
    match.insert(0, results[1])
match = list(dict.fromkeys(match))

search_priority = neighborhoods_venues_sorted.iloc[search_priority.index]
match = match_func(criterion)
main_score(search_priority)
match_score(search_priority)
total_score(search_priority)
    
i=0
for items in zip(search_priority['Neighborhood']):
    i=i+1
print(f"{i} Results Found!")

search_priority.style.applymap(df_style)

C:\Users\jhenn\anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\jhenn\anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\jhenn\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

20 Results Found!


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,Main Score,Number of Matches,Match Score,Total Score
29,South End,Italian Restaurant,American Restaurant,Wine Bar,French Restaurant,Wine Shop,Pet Store,Mexican Restaurant,Café,Gym / Fitness Center,Coffee Shop,Thai Restaurant,Park,Bakery,Japanese Restaurant,Gastropub,Mediterranean Restaurant,Pizza Place,Asian Restaurant,Bar,Gift Shop,1.000000,4,0.800000,0.900000
2,Back Bay,Clothing Store,Seafood Restaurant,Spa,Park,Italian Restaurant,Coffee Shop,Gym,Bakery,Wine Shop,New American Restaurant,Mediterranean Restaurant,Shopping Mall,Plaza,Salad Place,Steakhouse,Ice Cream Shop,Hotel,Sushi Restaurant,Gym / Fitness Center,Jewelry Store,0.750000,5,1.000000,0.875000
3,Beacon Hill,Coffee Shop,Italian Restaurant,Hotel,Historic Site,French Restaurant,Pizza Place,Seafood Restaurant,Park,Steakhouse,American Restaurant,Boat or Ferry,Theater,Hotel Bar,Scenic Lookout,Bakery,Bar,Gym / Fitness Center,Market,Restaurant,Gourmet Shop,0.900000,4,0.800000,0.850000
23,North End,Italian Restaurant,Park,Seafood Restaurant,Pizza Place,Hotel,Bakery,Coffee Shop,Historic Site,Sandwich Place,Yoga Studio,Aquarium,Beer Garden,Boat or Ferry,American Restaurant,Greek Restaurant,Gastropub,Church,Boutique,Ski Chalet,Skating Rink,1.000000,3,0.600000,0.800000
28,South Boston,Coffee Shop,Pizza Place,Park,Convenience Store,Italian Restaurant,Diner,Beach,New American Restaurant,Furniture / Home Store,Breakfast Spot,Bar,Pub,Donut Shop,Dive Bar,Sports Bar,American Restaurant,Restaurant,Bowling Alley,IT Services,Ice Cream Shop,0.750000,4,0.800000,0.775000
25,Roslindale,Pizza Place,Chinese Restaurant,Bakery,Italian Restaurant,Grocery Store,Convenience Store,Park,Coffee Shop,Sandwich Place,Liquor Store,Bar,Yoga Studio,American Restaurant,Frozen Yogurt Shop,Bus Line,Mexican Restaurant,Indian Restaurant,Restaurant,Ice Cream Shop,Big Box Store,0.750000,4,0.800000,0.775000
31,West End,Hotel,Italian Restaurant,Bakery,Park,Seafood Restaurant,Pizza Place,Historic Site,Coffee Shop,Science Museum,Yoga Studio,Bar,Deli / Bodega,Mexican Restaurant,French Restaurant,Gourmet Shop,Grocery Store,Scenic Lookout,Sandwich Place,Salon / Barbershop,Salad Place,0.900000,3,0.600000,0.750000
30,West Cambridge,Grocery Store,American Restaurant,Italian Restaurant,Restaurant,Auto Garage,BBQ Joint,Bakery,Liquor Store,Baseball Field,Park,Athletics & Sports,Chinese Restaurant,Sushi Restaurant,Cheese Shop,Other Great Outdoors,New American Restaurant,Lounge,Fish Market,Coffee Shop,Pizza Place,0.900000,3,0.600000,0.750000
27,Seaport,Seafood Restaurant,Hotel,Italian Restaurant,Steakhouse,Coffee Shop,Restaurant,Asian Restaurant,Gym,Brewery,French Restaurant,Mexican Restaurant,Bakery,Bar,Clothing Store,Art Museum,New American Restaurant,Mediterranean Restaurant,Scenic Lookout,Park,Deli / Bodega,0.900000,3,0.600000,0.750000
8,Chestnut Hill,Baseball Field,American Restaurant,Furniture / Home Store,Tennis Court,Italian Restaurant,Mobile Phone Shop,Grocery Store,Ice Cream Shop,Bank,Seafood Restaurant,Salon / Barbershop,Museum,Boutique,Coffee Shop,French Restaurant,Steakhouse,Forest,Food Truck,Clothing Store,Chinese Restaurant,0.750000,3,0.600000,0.675000


In [44]:
match

['Italian Restaurant',
 'Pizza Place',
 'Ice Cream Shop',
 'Gym',
 'Gym / Fitness Center',
 'Coffee Shop']

### Create df that includes Neighborhoods with ALL criterion

In [45]:
search_perf = search_priority.loc[search_priority['Number of Matches'] == 5]

rows = len(search_perf.index)
print(f"{rows} Results Found!")

if rows == 0:
    print("Criterion too specific")

search_perf.style.applymap(df_style)

2 Results Found!


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,Main Score,Number of Matches,Match Score,Total Score
2,Back Bay,Clothing Store,Seafood Restaurant,Spa,Park,Italian Restaurant,Coffee Shop,Gym,Bakery,Wine Shop,New American Restaurant,Mediterranean Restaurant,Shopping Mall,Plaza,Salad Place,Steakhouse,Ice Cream Shop,Hotel,Sushi Restaurant,Gym / Fitness Center,Jewelry Store,0.750000,5,1.000000,0.875000
1,Assembly Square,Clothing Store,Donut Shop,Mexican Restaurant,Coffee Shop,American Restaurant,Restaurant,Sporting Goods Shop,Video Store,Ice Cream Shop,Pizza Place,Park,Bar,Steakhouse,Brazilian Restaurant,Gym,Café,Sandwich Place,Seafood Restaurant,Italian Restaurant,Furniture / Home Store,0.200000,5,1.000000,0.600000


### Final Score Result for all Neighborhoods

In [48]:
match = match_func(criterion)
main_score(neighborhoods_venues_sorted)
match_score(neighborhoods_venues_sorted)
total_score(neighborhoods_venues_sorted)
neighborhoods_venues_sorted.to_excel('Neighborhood_Scored.xlsx')
neighborhoods_venues_sorted.style.applymap(df_style)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,Main Score,Number of Matches,Match Score,Total Score
29,South End,Italian Restaurant,American Restaurant,Wine Bar,French Restaurant,Wine Shop,Pet Store,Mexican Restaurant,Café,Gym / Fitness Center,Coffee Shop,Thai Restaurant,Park,Bakery,Japanese Restaurant,Gastropub,Mediterranean Restaurant,Pizza Place,Asian Restaurant,Bar,Gift Shop,1.000000,4,0.800000,0.900000
2,Back Bay,Clothing Store,Seafood Restaurant,Spa,Park,Italian Restaurant,Coffee Shop,Gym,Bakery,Wine Shop,New American Restaurant,Mediterranean Restaurant,Shopping Mall,Plaza,Salad Place,Steakhouse,Ice Cream Shop,Hotel,Sushi Restaurant,Gym / Fitness Center,Jewelry Store,0.750000,5,1.000000,0.875000
3,Beacon Hill,Coffee Shop,Italian Restaurant,Hotel,Historic Site,French Restaurant,Pizza Place,Seafood Restaurant,Park,Steakhouse,American Restaurant,Boat or Ferry,Theater,Hotel Bar,Scenic Lookout,Bakery,Bar,Gym / Fitness Center,Market,Restaurant,Gourmet Shop,0.900000,4,0.800000,0.850000
23,North End,Italian Restaurant,Park,Seafood Restaurant,Pizza Place,Hotel,Bakery,Coffee Shop,Historic Site,Sandwich Place,Yoga Studio,Aquarium,Beer Garden,Boat or Ferry,American Restaurant,Greek Restaurant,Gastropub,Church,Boutique,Ski Chalet,Skating Rink,1.000000,3,0.600000,0.800000
28,South Boston,Coffee Shop,Pizza Place,Park,Convenience Store,Italian Restaurant,Diner,Beach,New American Restaurant,Furniture / Home Store,Breakfast Spot,Bar,Pub,Donut Shop,Dive Bar,Sports Bar,American Restaurant,Restaurant,Bowling Alley,IT Services,Ice Cream Shop,0.750000,4,0.800000,0.775000
25,Roslindale,Pizza Place,Chinese Restaurant,Bakery,Italian Restaurant,Grocery Store,Convenience Store,Park,Coffee Shop,Sandwich Place,Liquor Store,Bar,Yoga Studio,American Restaurant,Frozen Yogurt Shop,Bus Line,Mexican Restaurant,Indian Restaurant,Restaurant,Ice Cream Shop,Big Box Store,0.750000,4,0.800000,0.775000
31,West End,Hotel,Italian Restaurant,Bakery,Park,Seafood Restaurant,Pizza Place,Historic Site,Coffee Shop,Science Museum,Yoga Studio,Bar,Deli / Bodega,Mexican Restaurant,French Restaurant,Gourmet Shop,Grocery Store,Scenic Lookout,Sandwich Place,Salon / Barbershop,Salad Place,0.900000,3,0.600000,0.750000
30,West Cambridge,Grocery Store,American Restaurant,Italian Restaurant,Restaurant,Auto Garage,BBQ Joint,Bakery,Liquor Store,Baseball Field,Park,Athletics & Sports,Chinese Restaurant,Sushi Restaurant,Cheese Shop,Other Great Outdoors,New American Restaurant,Lounge,Fish Market,Coffee Shop,Pizza Place,0.900000,3,0.600000,0.750000
27,Seaport,Seafood Restaurant,Hotel,Italian Restaurant,Steakhouse,Coffee Shop,Restaurant,Asian Restaurant,Gym,Brewery,French Restaurant,Mexican Restaurant,Bakery,Bar,Clothing Store,Art Museum,New American Restaurant,Mediterranean Restaurant,Scenic Lookout,Park,Deli / Bodega,0.900000,3,0.600000,0.750000
8,Chestnut Hill,Baseball Field,American Restaurant,Furniture / Home Store,Tennis Court,Italian Restaurant,Mobile Phone Shop,Grocery Store,Ice Cream Shop,Bank,Seafood Restaurant,Salon / Barbershop,Museum,Boutique,Coffee Shop,French Restaurant,Steakhouse,Forest,Food Truck,Clothing Store,Chinese Restaurant,0.750000,3,0.600000,0.675000


## Map all Italian Restaurants in Boston

In [28]:
Boston_Italian = Boston_venues.loc[Boston_venues['Venue Category'] == 'Italian Restaurant']
Boston_Italian.drop(['Neighborhood Latitude', 'Neighborhood Longitude'], axis=1, inplace=True)
fixed_columns = [Boston_Italian.columns[1]] + list(Boston_Italian.columns[2:4]) + [Boston_Italian.columns[0]] + [Boston_Italian.columns[-1]]
Boston_Italian = Boston_Italian[fixed_columns]

C:\Users\jhenn\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [29]:
Boston_Italian

,Venue,Venue Latitude,Venue Longitude,Neighborhood,Venue Category
84,Maggiano's Little Italy,42.350966,-71.068695,Chinatown/Leather District,Italian Restaurant
111,Osteria Rustico,42.364397,-71.059870,West End,Italian Restaurant
121,Scampo at The Liberty Hotel,42.361937,-71.070513,West End,Italian Restaurant
124,Grotto,42.359915,-71.062807,West End,Italian Restaurant
177,Aria,42.363286,-71.055002,West End,Italian Restaurant
...,...,...,...,...,...
2486,Sportello,42.350519,-71.048687,Seaport,Italian Restaurant
2510,Sapóre Ristorante,42.348126,-71.039863,Seaport,Italian Restaurant
2518,Buco Trattoria,42.349923,-71.049393,Seaport,Italian Restaurant
2524,Tuscan Kitchen,42.352829,-71.047298,Seaport,Italian Restaurant


In [30]:
# create map of Boston using latitude and longitude values
map_Italian = folium.Map(location=[42.3261, -71.07], zoom_start=12)

# add Restaurant markers to map
for lat, lng, venue, neighborhood in zip(Boston_Italian['Venue Latitude'], Boston_Italian['Venue Longitude'], Boston_Italian['Venue'], Boston_Italian['Neighborhood']):
    label = f'{venue}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Italian)
    
# add Neighborhood markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df.index):
    label = f'{neighborhood}'
    label = folium.Popup(label, parse_html=True)
    
    folium.Circle(
        [lat, lng],
        radius=1000,
        popup=label).add_to(map_Italian)
    
map_Italian

## Map Top Neighborhood

In [38]:
latitude = []
longitude = []

for i in range(0,3):
    
    Hood = Boston_venues.loc[Boston_venues['Neighborhood'] == neighborhoods_venues_sorted.iloc[i]['Neighborhood']]
    if i == 0:
        Top_Hood = Boston_venues.loc[Boston_venues['Neighborhood'] == neighborhoods_venues_sorted.iloc[i]['Neighborhood']]
    elif i == 1:
        Two_Hood = Boston_venues.loc[Boston_venues['Neighborhood'] == neighborhoods_venues_sorted.iloc[i]['Neighborhood']]
    elif i == 2:
        Three_Hood = Boston_venues.loc[Boston_venues['Neighborhood'] == neighborhoods_venues_sorted.iloc[i]['Neighborhood']]
        
        
    latitude.append(Hood.iloc[0]['Neighborhood Latitude'])
    longitude.append(Hood.iloc[0]['Neighborhood Longitude'])
    
Top_Hood = pd.concat([Top_Hood, Two_Hood, Three_Hood])
Top_Hood

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
200,South End,42.33780,-71.07060,Stella,42.339736,-71.072574,Italian Restaurant
201,South End,42.33780,-71.07060,anoush’ella,42.338869,-71.073843,Mediterranean Restaurant
202,South End,42.33780,-71.07060,Flour Bakery & Cafe,42.338447,-71.074299,Bakery
203,South End,42.33780,-71.07060,The Gallows,42.341262,-71.070135,Gastropub
204,South End,42.33780,-71.07060,Boston Flower Exchange,42.337819,-71.066743,Flower Shop
...,...,...,...,...,...,...,...
1412,Beacon Hill,42.35976,-71.06775,Good Will Hunting Bench,42.354391,-71.070673,General Entertainment
1413,Beacon Hill,42.35976,-71.06775,Old South Meeting House,42.357183,-71.058647,Historic Site
1414,Beacon Hill,42.35976,-71.06775,Cutler Majestic Theatre,42.351545,-71.064850,Theater
1415,Beacon Hill,42.35976,-71.06775,Faneuil Hall Marketplace,42.359978,-71.056410,Historic Site


In [39]:
# create map of Boston using latitude and longitude values
from statistics import mean
map_Top_Hood = folium.Map(location=[mean(latitude), mean(longitude)], zoom_start=13)

# add Restaurant markers to map
for lat, lng, venue in zip(Top_Hood['Venue Latitude'], Top_Hood['Venue Longitude'], Top_Hood['Venue']):
    label = f'{venue}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Top_Hood)
    
# add Neighborhood markers to map
for lat, long in zip(latitude, longitude):
    
    df1 = df.loc[df['Latitude'] == lat]
    label = df1.index[0]
    label = folium.Popup(label, parse_html=True)
    
    if lat == latitude[0]:
        folium.Circle(
            [lat, long],
            radius=1000,
            color='red',
            popup=label).add_to(map_Top_Hood)
    else:
        folium.Circle(
            [lat, long],
            radius=1000,
            color='blue',
            popup=label).add_to(map_Top_Hood)

map_Top_Hood